Este Notebook contém minhas anotações, e é baseado no original disponível em: https://www.kaggle.com/code/phelpsmemo/cap394-exemplo-tidy-step-by-step-python

<br>

<h1>CAP 394: Exemplo de criação de dados Tidy</h1>

- **Instrutores**: Adriano Almeida, Felipe Carvalho e Felipe Menino

- **Descrição**: Existem muitas maneiras de realizar a *tidyficação* de um conjunto de dados. Neste exemplo, faz-se a apresentação, passo a passo, de uma dessas maneiras, onde todas as necessidades e propriedades de dados **Tidy** são construídas em etapas. Para saber mais sobre dados **Tidy** e a teoria subjacente a esta forma de organização dos dados, consulte a página [Tidy data](https://r4ds.had.co.nz/tidy-data.html).


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../input/cap-394-exercicio-de-analise-de-dados/treino.csv")

In [3]:
df

,id,b1,b2,b3,b4,b5,b6,b7,b8,b9,...,pred_minus_obs_S_b3,pred_minus_obs_S_b4,pred_minus_obs_S_b5,pred_minus_obs_S_b6,pred_minus_obs_S_b7,pred_minus_obs_S_b8,pred_minus_obs_S_b9,label,X,Y
0,454,54,28,51,93,56,100,80,24,55,...,-3.99,-23.22,-0.94,-4.30,-23.22,-1.69,-4.55,s,140.269397,36.897564
1,457,54,29,51,88,51,91,95,24,55,...,-4.09,-22.24,-1.01,-4.93,-17.81,-0.53,-2.98,s,140.246424,36.403438
2,261,59,30,52,90,54,93,80,26,58,...,-5.61,-27.73,-1.15,-5.74,-21.81,-2.39,-5.07,s,140.080862,35.906891
3,8,53,27,49,95,49,92,63,25,54,...,-5.95,-39.27,-2.13,-8.73,-30.73,-2.42,-5.58,s,140.266256,36.585268
4,478,58,40,65,100,58,100,106,26,57,...,-4.85,-21.15,-1.00,-4.84,-17.00,-0.91,-3.38,d,140.151532,36.062363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,292,55,28,50,92,50,93,70,25,56,...,-1.88,-21.14,-0.70,-3.87,-16.79,-0.71,-3.00,s,140.494570,36.237215
205,409,59,72,98,91,85,125,118,47,87,...,-3.30,-17.80,-1.12,-3.94,-17.20,-1.04,-2.96,o,140.250817,36.544129
206,23,68,28,54,110,50,98,95,27,64,...,-3.01,-16.95,-0.94,-5.06,-7.59,0.24,-2.12,h,140.391184,36.580138
207,453,60,56,71,94,64,99,100,26,57,...,-5.78,-17.65,-1.26,-4.78,-17.60,-1.67,-3.90,d,140.081026,36.055888


Neste exemplo, o objetivo é fazer com que o conjunto de dados apresentado acima, tenha os seguintes atributos:

| id 	| X 	| Y 	| date 	| spectral_band 	| spectral_band_value 	| weighted_spectral_band_value 	| class_weighted 	| label 	|
|:--:	|:-:	|:-:	|:----:	|:-------------:	|:-------------------:	|:----------------------------:	|:--------------:	|:-----:	|

### 1. Separando as propriedades

**Definindo as características gerais que serão aplicadas nos dados**

In [4]:
scene_dates = ["09/2010", "03/2011", "05/2011"]
attribute_names = ["red", "green", "nir", "label", "X", "Y"]

**Definindo função auxiliar**

In [5]:
def transform_by_date(ldf, lscene_date, attrname, varname, valname):
    ldf = ldf.copy()
    ldf.columns = attrname
    
    return ldf.assign(date = lscene_date).melt(id_vars = ["date", "X", "Y", "label"], var_name = varname, value_name=valname)

**Separando as colunas `bands` por data**

In [6]:
bands_d1 = df.filter(regex="^b[1-3]$|(X|Y|label)")

bands_d2 = df.filter(regex="^b[4-6]|(X|Y|label)$")

bands_d3 = df.filter(regex="^b[7-9]|(X|Y|label)$")

**Separando as colunas `bands_weighted (S)` por data**

In [7]:
bands_d1_weighted_s = df.filter(regex="^pred_minus_obs_S_b[1-3]|(X|Y|label)$")

bands_d2_weighted_s = df.filter(regex="^pred_minus_obs_S_b[4-6]|(X|Y|label)$")

bands_d3_weighted_s = df.filter(regex="^pred_minus_obs_S_b[7-9]|(X|Y|label)$")

**Separando as colunas `bands_weighted (H)` por data**

In [8]:
bands_d1_weighted_h = df.filter(regex="^pred_minus_obs_H_b[1-3]|(X|Y|label)$")

bands_d2_weighted_h = df.filter(regex="^pred_minus_obs_H_b[4-6]|(X|Y|label)$")

bands_d3_weighted_h = df.filter(regex="^pred_minus_obs_H_b[7-9]|(X|Y|label)$")

### 2. Transformando colunas (em atributos)

**Transformando os atributos `bands`**

*09/2010*

In [9]:
bands_d1 = transform_by_date(bands_d1, scene_dates[0], attribute_names, "spectral_band", "spectral_band_value")
bands_d1

,date,X,Y,label,spectral_band,spectral_band_value
0,09/2010,140.269397,36.897564,s,red,54
1,09/2010,140.246424,36.403438,s,red,54
2,09/2010,140.080862,35.906891,s,red,59
3,09/2010,140.266256,36.585268,s,red,53
4,09/2010,140.151532,36.062363,d,red,58
...,...,...,...,...,...,...
622,09/2010,140.494570,36.237215,s,nir,50
623,09/2010,140.250817,36.544129,o,nir,98
624,09/2010,140.391184,36.580138,h,nir,54
625,09/2010,140.081026,36.055888,d,nir,71


*03/2011*

In [10]:
bands_d2 = transform_by_date(bands_d2, scene_dates[1], attribute_names, "spectral_band", "spectral_band_value")
bands_d2

,date,X,Y,label,spectral_band,spectral_band_value
0,03/2011,140.269397,36.897564,s,red,93
1,03/2011,140.246424,36.403438,s,red,88
2,03/2011,140.080862,35.906891,s,red,90
3,03/2011,140.266256,36.585268,s,red,95
4,03/2011,140.151532,36.062363,d,red,100
...,...,...,...,...,...,...
622,03/2011,140.494570,36.237215,s,nir,93
623,03/2011,140.250817,36.544129,o,nir,125
624,03/2011,140.391184,36.580138,h,nir,98
625,03/2011,140.081026,36.055888,d,nir,99


*05/2011*

In [11]:
bands_d3 = transform_by_date(bands_d3, scene_dates[2], attribute_names, "spectral_band", "spectral_band_value")
bands_d3

,date,X,Y,label,spectral_band,spectral_band_value
0,05/2011,140.269397,36.897564,s,red,80
1,05/2011,140.246424,36.403438,s,red,95
2,05/2011,140.080862,35.906891,s,red,80
3,05/2011,140.266256,36.585268,s,red,63
4,05/2011,140.151532,36.062363,d,red,106
...,...,...,...,...,...,...
622,05/2011,140.494570,36.237215,s,nir,56
623,05/2011,140.250817,36.544129,o,nir,87
624,05/2011,140.391184,36.580138,h,nir,64
625,05/2011,140.081026,36.055888,d,nir,57


**Transformando os atributos `bands_weighted (S)`**

*09/2010*

In [12]:
bands_d1_weighted_s = transform_by_date(bands_d1_weighted_s, scene_dates[0], attribute_names, "spectral_band", "weighted_spectral_band_value")

bands_d1_weighted_s["class_weighted"] = "s"
bands_d1_weighted_s

,date,X,Y,label,spectral_band,weighted_spectral_band_value,class_weighted
0,09/2010,140.269397,36.897564,s,red,-24.52,s
1,09/2010,140.246424,36.403438,s,red,-19.03,s
2,09/2010,140.080862,35.906891,s,red,-21.04,s
3,09/2010,140.266256,36.585268,s,red,-25.65,s
4,09/2010,140.151532,36.062363,d,red,-18.31,s
...,...,...,...,...,...,...,...
622,09/2010,140.494570,36.237215,s,nir,-1.88,s
623,09/2010,140.250817,36.544129,o,nir,-3.30,s
624,09/2010,140.391184,36.580138,h,nir,-3.01,s
625,09/2010,140.081026,36.055888,d,nir,-5.78,s


*03/2011*

In [13]:
bands_d2_weighted_s = transform_by_date(bands_d2_weighted_s, scene_dates[1], attribute_names, "spectral_band", "weighted_spectral_band_value")

bands_d2_weighted_s["class_weighted"] = "s"
bands_d2_weighted_s

,date,X,Y,label,spectral_band,weighted_spectral_band_value,class_weighted
0,03/2011,140.269397,36.897564,s,red,-23.22,s
1,03/2011,140.246424,36.403438,s,red,-22.24,s
2,03/2011,140.080862,35.906891,s,red,-27.73,s
3,03/2011,140.266256,36.585268,s,red,-39.27,s
4,03/2011,140.151532,36.062363,d,red,-21.15,s
...,...,...,...,...,...,...,...
622,03/2011,140.494570,36.237215,s,nir,-3.87,s
623,03/2011,140.250817,36.544129,o,nir,-3.94,s
624,03/2011,140.391184,36.580138,h,nir,-5.06,s
625,03/2011,140.081026,36.055888,d,nir,-4.78,s


*05/2011*

In [14]:
bands_d3_weighted_s = transform_by_date(bands_d3_weighted_s, scene_dates[2], attribute_names, "spectral_band", "weighted_spectral_band_value")

bands_d3_weighted_s["class_weighted"] = "s"
bands_d3_weighted_s

,date,X,Y,label,spectral_band,weighted_spectral_band_value,class_weighted
0,05/2011,140.269397,36.897564,s,red,-23.22,s
1,05/2011,140.246424,36.403438,s,red,-17.81,s
2,05/2011,140.080862,35.906891,s,red,-21.81,s
3,05/2011,140.266256,36.585268,s,red,-30.73,s
4,05/2011,140.151532,36.062363,d,red,-17.00,s
...,...,...,...,...,...,...,...
622,05/2011,140.494570,36.237215,s,nir,-3.00,s
623,05/2011,140.250817,36.544129,o,nir,-2.96,s
624,05/2011,140.391184,36.580138,h,nir,-2.12,s
625,05/2011,140.081026,36.055888,d,nir,-3.90,s


**Transformando os atributos `bands_weighted (H)`**

*09/2010*

In [15]:
bands_d1_weighted_h = transform_by_date(bands_d1_weighted_h, scene_dates[0], attribute_names, "spectral_band", "weighted_spectral_band_value")

bands_d1_weighted_h["class_weighted"] = "h"
bands_d1_weighted_h

,date,X,Y,label,spectral_band,weighted_spectral_band_value,class_weighted
0,09/2010,140.269397,36.897564,s,red,64.88,h
1,09/2010,140.246424,36.403438,s,red,56.28,h
2,09/2010,140.080862,35.906891,s,red,56.58,h
3,09/2010,140.266256,36.585268,s,red,66.97,h
4,09/2010,140.151532,36.062363,d,red,51.79,h
...,...,...,...,...,...,...,...
622,09/2010,140.494570,36.237215,s,nir,46.99,h
623,09/2010,140.250817,36.544129,o,nir,-6.63,h
624,09/2010,140.391184,36.580138,h,nir,43.75,h
625,09/2010,140.081026,36.055888,d,nir,21.52,h


*03/2011*

In [16]:
bands_d2_weighted_h = transform_by_date(bands_d2_weighted_h, scene_dates[1], attribute_names, "spectral_band", "weighted_spectral_band_value")

bands_d2_weighted_h["class_weighted"] = "h"
bands_d2_weighted_h

,date,X,Y,label,spectral_band,weighted_spectral_band_value,class_weighted
0,03/2011,140.269397,36.897564,s,red,6.74,h
1,03/2011,140.246424,36.403438,s,red,10.16,h
2,03/2011,140.080862,35.906891,s,red,12.77,h
3,03/2011,140.266256,36.585268,s,red,8.08,h
4,03/2011,140.151532,36.062363,d,red,-1.00,h
...,...,...,...,...,...,...,...
622,03/2011,140.494570,36.237215,s,nir,-31.60,h
623,03/2011,140.250817,36.544129,o,nir,-66.95,h
624,03/2011,140.391184,36.580138,h,nir,-34.37,h
625,03/2011,140.081026,36.055888,d,nir,-40.28,h


*05/2011*

In [17]:
bands_d3_weighted_h = transform_by_date(bands_d3_weighted_h, scene_dates[2], attribute_names, "spectral_band", "weighted_spectral_band_value")

bands_d3_weighted_h["class_weighted"] = "h"
bands_d3_weighted_h

,date,X,Y,label,spectral_band,weighted_spectral_band_value,class_weighted
0,05/2011,140.269397,36.897564,s,red,0.47,h
1,05/2011,140.246424,36.403438,s,red,-21.37,h
2,05/2011,140.080862,35.906891,s,red,-1.30,h
3,05/2011,140.266256,36.585268,s,red,19.78,h
4,05/2011,140.151532,36.062363,d,red,-32.17,h
...,...,...,...,...,...,...,...
622,05/2011,140.494570,36.237215,s,nir,-1.15,h
623,05/2011,140.250817,36.544129,o,nir,-32.43,h
624,05/2011,140.391184,36.580138,h,nir,-9.84,h
625,05/2011,140.081026,36.055888,d,nir,-2.09,h


## Juntando todas as peças

**Juntando os atributos `bands`**

In [18]:
bands = pd.concat([bands_d1, bands_d2, bands_d3])
bands

,date,X,Y,label,spectral_band,spectral_band_value
0,09/2010,140.269397,36.897564,s,red,54
1,09/2010,140.246424,36.403438,s,red,54
2,09/2010,140.080862,35.906891,s,red,59
3,09/2010,140.266256,36.585268,s,red,53
4,09/2010,140.151532,36.062363,d,red,58
...,...,...,...,...,...,...
622,05/2011,140.494570,36.237215,s,nir,56
623,05/2011,140.250817,36.544129,o,nir,87
624,05/2011,140.391184,36.580138,h,nir,64
625,05/2011,140.081026,36.055888,d,nir,57


**Juntando os atributos `bands_weighted (S)`**

In [19]:
bands_weighted_s = pd.concat([bands_d1_weighted_s, bands_d2_weighted_s, bands_d3_weighted_s])
bands_weighted_s

,date,X,Y,label,spectral_band,weighted_spectral_band_value,class_weighted
0,09/2010,140.269397,36.897564,s,red,-24.52,s
1,09/2010,140.246424,36.403438,s,red,-19.03,s
2,09/2010,140.080862,35.906891,s,red,-21.04,s
3,09/2010,140.266256,36.585268,s,red,-25.65,s
4,09/2010,140.151532,36.062363,d,red,-18.31,s
...,...,...,...,...,...,...,...
622,05/2011,140.494570,36.237215,s,nir,-3.00,s
623,05/2011,140.250817,36.544129,o,nir,-2.96,s
624,05/2011,140.391184,36.580138,h,nir,-2.12,s
625,05/2011,140.081026,36.055888,d,nir,-3.90,s


**Juntando os atributos `bands_weighted (H)`**

In [20]:
bands_weighted_h = pd.concat([bands_d1_weighted_h, bands_d2_weighted_h, bands_d3_weighted_h])
bands_weighted_h

,date,X,Y,label,spectral_band,weighted_spectral_band_value,class_weighted
0,09/2010,140.269397,36.897564,s,red,64.88,h
1,09/2010,140.246424,36.403438,s,red,56.28,h
2,09/2010,140.080862,35.906891,s,red,56.58,h
3,09/2010,140.266256,36.585268,s,red,66.97,h
4,09/2010,140.151532,36.062363,d,red,51.79,h
...,...,...,...,...,...,...,...
622,05/2011,140.494570,36.237215,s,nir,-1.15,h
623,05/2011,140.250817,36.544129,o,nir,-32.43,h
624,05/2011,140.391184,36.580138,h,nir,-9.84,h
625,05/2011,140.081026,36.055888,d,nir,-2.09,h


**Juntando os atributos `bands_weighted (H)`, `bands_weighted (S)` e `bands`**

In [21]:
mydf = pd.concat([
    
    pd.merge(left = bands, right=bands_weighted_s),
    
    pd.merge(left = bands, right=bands_weighted_h)
    
])
mydf

,date,X,Y,label,spectral_band,spectral_band_value,weighted_spectral_band_value,class_weighted
0,09/2010,140.269397,36.897564,s,red,54,-24.52,s
1,09/2010,140.246424,36.403438,s,red,54,-19.03,s
2,09/2010,140.080862,35.906891,s,red,59,-21.04,s
3,09/2010,140.266256,36.585268,s,red,53,-25.65,s
4,09/2010,140.151532,36.062363,d,red,58,-18.31,s
...,...,...,...,...,...,...,...,...
1876,05/2011,140.494570,36.237215,s,nir,56,-1.15,h
1877,05/2011,140.250817,36.544129,o,nir,87,-32.43,h
1878,05/2011,140.391184,36.580138,h,nir,64,-9.84,h
1879,05/2011,140.081026,36.055888,d,nir,57,-2.09,h
